# YOLO Training Notebook

## Setup Instance

The first step after starting the `training.ipynb` notebook is to run the `setup.sh` script in the first cell. Note: `setup.sh` must have the executable permission set i.e., `chmod +x setup.sh`.

This script sets up the instance so that it can use Darknet to train a model.

- Clone Darknet Repo
- Edit Darknet Makefile
- Build Darknet
- Download YOLOv4-tiny weights
- Copy YOLOv4-tiny config

In [ ]:
!./setup.sh

## Setup Training

In [ ]:
from convert_model.convert import convert_darknet_to_tf
from utils.config import update_config
from utils.data_split import yolo_obj_detection_setup
from glob import glob
import os
import shutil

* Set paths

In [ ]:
base_folder = f"{os.getcwd()}/"
labels = ["car", "truck", "bus"]  # change labels to suit annotated data
cfg_file = f"{base_folder}model_files/yolov4-tiny.cfg"
weights_file = f"{base_folder}model_files/yolov4-tiny.conv.29"
obj_data = f"{base_folder}model_files/obj.data"
obj_test_data = f"{base_folder}model_files/obj_test.data"

* Split data

In [ ]:
yolo_obj_detection_setup(folder=base_folder, labels=labels, train_pct=.6, test_pct=.1, val_pct=.3)

* Update config

In [ ]:
update_config(cfg_file, labels=labels, input_size=416, subdivisions=4)

## Train Model

In [ ]:
%cd $base_folder"darknet/"

In [ ]:
%%capture
!./darknet detector train $obj_data $cfg_file $weights_file -dont_show -ext_output -map > $base_folder"model_files/log_training.txt"

In [ ]:
for weights in glob(f"{base_folder}darknet/backup/*.weights"):
    fname = weights.split("/")[-1]
    shutil.copy(weights, f"{base_folder}model_files/{fname}")

## Evaluate Model

In [ ]:
%%capture
best_weights = f"{base_folder}/darknet/backup/yolov4-tiny_best.weights"
!./darknet detector map $obj_test_data $cfg_file $best_weights -points 0  > $base_folder"model_files/log_evaluation.txt"

## Convert Model

**Note:** need to restart kernel after each conversion i.e., if converting to multiple formats.

In [ ]:
%cd $base_folder

In [ ]:
convert_darknet_to_tf(f"{base_folder}model_files/", framework="tflite", tiny=True)